PopDanceSet Data Visualization: 

Everyone has their own preferences, so you can freely decide which dances you want and which you do not by rendering the dances in the video foramt. Furthermore, you can even create your own dance dataset based on PopDanceSet.

In [ ]:
import pickle
from vis import SMPLSkeleton
from vis import skeleton_render
import torch
import os
from pytorch3d.transforms import (RotateAxisAngle, axis_angle_to_quaternion,
                                  quaternion_multiply,
                                  quaternion_to_axis_angle)

def view_pickle_file(file_path):
    try:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        
        print("The content of Pickle files:")
        
        pos = data["pos"]
        nframes,_ = pos.shape
        q = data["q"].reshape(nframes,-1,3)        
        
    except Exception as e:
        print("Failed to read the .pkl file:", str(e))
        
    return pos, q

directory = 'data/train/motions'
files = os.listdir(directory)

for file in files:
    if file.endswith('.pkl'):
        new_file_name = file.split('.')[0]
        new_file_path = f'data/train/motions_sliced/{new_file_name}_slice0.pkl'
        pos , q = view_pickle_file(new_file_path)

        smpl = SMPLSkeleton()
        pos = torch.from_numpy(pos).unsqueeze(0)
        q = torch.from_numpy(q).unsqueeze(0)

        root_q = q[:, :, :1, :]  
        root_q_quat = axis_angle_to_quaternion(root_q)
        rotation = torch.Tensor(
            [0.7071068, -0.7071068, 0, 0]
        )  
        root_q_quat = quaternion_multiply(rotation, root_q_quat)
        root_q = quaternion_to_axis_angle(root_q_quat)
        q[:, :, :1, :] = root_q

        pos_rotation = RotateAxisAngle(-90, axis="X", degrees=True)
        pos = pos_rotation.transform_points(
            pos
        )
        pos[:, :, 2] += 1
        pos[:, :, 1] -= 5

        full_pose = (smpl.forward(q,pos).detach().cpu().numpy())
        # print(full_pose.shape)
        render_dir = 'renders'
        exp_name = 'test_1'
        render_out = os.path.join(render_dir, "train_" + exp_name)
        epoch = 0
        file_name = os.path.basename(new_file_path)
        name = os.path.splitext(file_name)[0]
        skeleton_render(
            full_pose[0],
            epoch = f"{epoch}",
            out = render_out,
            # name = "BV1dY411u7fD_Clip_2_slice0",
            name = name,
            sound = False
        )